 EPIC, the Extended PID Configurator, Ihr vertrauenswürdiger Konfigurator :)
V0.1 := Initial release.

#Writed according PEP8 style guide
#https://realpython.com/python-pep8/

In [1684]:
#Libraries import
import os
import time
import datetime
import pandas as pd
import numpy as np
from numpy import random
from enum import Enum

##Version information
print(f"Pandas version: {pd.__version__}") #Pandas version number
print(f"Numpy version: {np.__version__}") #numpy version number

pd.options.display.float_format = '{:.3f}'.format



Pandas version: 1.4.3
Numpy version: 1.23.1


In [1685]:
#Global variables
import_mode = True # import mode (True/False) - Imports an csv from PID
verbose_mode = False # verbose mode (True/False)
debug_mode = False # debug mode (True/False)
random_mode = False # random data generation  (True/False)
simulation_mode = False # simulation (True/False)

In [1686]:
#Generate standard templated
generate_tag_number_definitions = False #Every element has a description and a unique number for software modules
#generate_element_list_alarm_matrix = False #Every element has a unique number of alarms
#generate_element_list_hardware_matrix = False #All possible combinations of elements have a unique IO

In [1687]:
#Data generation
generate_element_list = False
generate_functional_matrix = False
generate_alarm_matrix = False
generate_hardware_config = False
generate_symbolic   = False
generate_s_plant    = False
generate_s_line = False
generate_flow_chart = False #--> GUI with active modes and phases, start condition [active modes list ] as bool (true false). Generarte excel which generete flow chart.


In [1688]:
if import_mode: 
    ##Change working directory to temp/epic
    os.chdir('C://temp/epic/')
    cwd = os.getcwd()
    print('Current Working Directory is: ', cwd)
    


Current Working Directory is:  C:\temp\epic


In [1689]:
#print(os.listdir(cwd)) #List directory

#Random_mode:
Generate aleatories PID numbers and symbols.



In [1690]:
#Generate list of X random numbers from 1.100 - 9.999:
if random_mode:
    random_PID_numbers=list(random.randint(1100,10000, size=(1000)))

    if verbose_mode:    print(random_PID_numbers)

In [1691]:
#Generate PID symbol
if random_mode:
    random_PID_symbols = list(random.choice(['B', 'C', 'D', 'EC', 'F', 'FA-', 'FIA+', 'FICA+', 'FSA-', 'GIA', 'GSA+', 'GSA-', 'GZA-', 'H', 'K', 'L', 'LSA-', 'M',
    'MIA±', 'NC', 'P', 'PDI', 'PDICA±', 'PDIS', 'PI', 'PSA-', 'QIA+', 'QIA-', 'QSA+', 'S', 'SC', 'SIA±', 'T', 'TI', 'TIC', 'TICA±', 'TSA+',
    'V', 'W', 'WICA±', 'XA', 'Y', 'YC'], size=(1000)))

    if verbose_mode:    print(random_PID_symbols)
    if debug_mode:      print(type(random_PID_symbols))

PID.xlsx

In [1692]:
#Load PID data
#df_data_excel = pd.read_excel('PID.xlsx', index_col='Prop_PID_Number')
df_data_excel = pd.read_excel('PID.xlsx')
#df_data_excel = pd.read_excel('PID_SVEN.xlsx')
#df_data_excel = pd.read_excel('PID_NINIO.xlsx')
df_data_excel = df_data_excel.sort_values('Prop_PID_Element') #Sorting by Element we get elements ordered and ready to compare
if verbose_mode: print(df_data_excel)

In [1693]:
#Remove ShapeKey from the dataframe
del df_data_excel['ShapeKey']

In [1694]:
# dropping duplicate values
df_data_excel.drop_duplicates(inplace=True)

Drop elements like filters, catalizators, etc

In [1695]:
banned_elements = ['B','F', 'C', 'D', 'S', 'W', 'Z']
df_data_excel = df_data_excel[~df_data_excel['Prop_PID_Element'].isin(banned_elements)].reindex()

AutomationCode.xlsx

In [1696]:
#Load Automation Code excel file
df_automation_code = pd.read_excel('AutomationCode.xlsx')
if debug_mode: df_automation_code.head()

TND.xlsx

In [1697]:
#Load Tag Name Definition
tnd_pid = pd.read_excel('TND.xlsx')
#tnd_pid = pd.read_excel('TND.xlsx',index_col='PID')
if debug_mode: tnd_pid

Functions definitions

In [1698]:
# Def of Lookup as in Microsoft Excel.
def xlookup(lookup_value, lookup_array, return_array, if_not_found:str = ''):
    match_value = return_array.loc[lookup_array == lookup_value]
    if match_value.empty:
        return f'"{lookup_value}" not found!' if if_not_found == '' else if_not_found

    else:
        return match_value.tolist()[0]

In [1699]:
# Lookup for PID number: 1.840 or 2.840 return the same definition: Diff. pressure sensor
def xlookupPID(lookup_value, lookup_array, return_array, if_not_found:str = ''):
    lookup_value = str("%.3f" % lookup_value)
    lookup_value = 'Y' + lookup_value[1:]
    match_value = return_array.loc[lookup_array == lookup_value]
    if match_value.empty:
        return f'"{lookup_value}" not found!' if if_not_found == '' else if_not_found

    else:
        return match_value.tolist()[0]

In [1700]:
#Group by PID number
element_list = df_data_excel.groupby(['Prop_PID_Number']).agg(list).copy()

Group Filter
#Drop compound elements which are not part of the software list
#Drop elements like filters, catalizators, etc

In [1701]:
#Group filter: 

banned_elements = [['PI', 'V'],['ANOTHER', 'COMBO']]
element_list = element_list[~element_list['Prop_PID_Element'].isin(banned_elements)].reindex()

"""
for items in element_list.index:
    if element_list.at[items, 'Prop_PID_Element'] == ['PI', 'V']:
        print("Deleted a Pressure indicator in: ",items)
        element_list.drop(items,axis=0,inplace=True)
"""

'\nfor items in element_list.index:\n    if element_list.at[items, \'Prop_PID_Element\'] == [\'PI\', \'V\']:\n        print("Deleted a Pressure indicator in: ",items)\n        element_list.drop(items,axis=0,inplace=True)\n'

Reset the dataframe index. Objects can be easily accessed

In [1702]:
##Add tag number definition - reset index first, then use xlookup function
element_list = element_list.reset_index()

Deletion of elements that are non electronics, like manual valves.

In [1703]:
#Deletion of manual valves, as we have no other information in the PID
#TODO: import from a file banned element
#TODO: save deleted elements in a file

banned_elements = [1.654,1.656,1.850,1.863]
element_list = element_list[~element_list['Prop_PID_Number'].isin(banned_elements)].reindex()

In [1704]:
#Add Tag Number definition
element_list['Tag_Number_Definition'] = element_list['Prop_PID_Number'].apply(xlookupPID, args = (tnd_pid['PID'], tnd_pid['Description']));

In [1705]:
#Float to string
#element_list['Prop_PID_Number'] = element_list['Prop_PID_Number'].astype(str)

In [1706]:
#Add columns
element_list_sw_modules = ['SW_Module','SW_Module_Nr','Connection1','Connection1_Nr','Connection2','Connection2_Nr', 'Connection3','Connection3_Nr','Connection4','Connection4_Nr','Element_Description']
element_list = element_list.assign(**dict.fromkeys(element_list_sw_modules))

Add elements that has two software main modules, as 1.825 Leak valve (motor,reg.limit) and 1.825 Leak (leak, limit)
SIS module and gasket are now possible to configure

index+1     1.825   [Leak] [Limit_hi_lo] 

In [1707]:
#TODO: use patterns (Y.XXX) instead
if (1.655 and 1.680) in element_list.Prop_PID_Number.values: 
    dicc_AHU01 = {'Prop_PID_Number': 'AHU01', 'Prop_PID_Element': 'AHU01' , 'Tag_Number_Definition': 'Leader/Follower AHU 01 1.800 - (1.655-1.680'}
    element_list = pd.concat([element_list, pd.DataFrame([dicc_AHU01])])

if (1.800) in element_list.Prop_PID_Number.values:
    dicc_1800 = {'Prop_PID_Number': '1.800_precon', 'Prop_PID_Element': '[702] [17°C < 28°C]' , 'Tag_Number_Definition': 'Temperature sensor'}
    #element_list = element_list.append(dicc_1800, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1800])])

    dicc_1800 = {'Prop_PID_Number': '1.800_deco', 'Prop_PID_Element': '[118] [17°C < 28°C]' , 'Tag_Number_Definition': 'Temperature sensor'}
    #element_list = element_list.append(dicc_1800, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1800])])

    dicc_1800 = {'Prop_PID_Number': '1.800_pro', 'Prop_PID_Element': '[114] [15°C < 21°C]' , 'Tag_Number_Definition': 'Temperature sensor'}
    #element_list = element_list.append(dicc_1800, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1800])])

    dicc_1800 = {'Prop_PID_Number': '1.800_aer1', 'Prop_PID_Element': '[708] [30°C < 40°C]' , 'Tag_Number_Definition': 'Temperature sensor'}
    #element_list = element_list.append(dicc_1800, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1800])])

if (1.810) in element_list.Prop_PID_Number.values:
    dicc_1810 = {'Prop_PID_Number': '1.810_pro', 'Prop_PID_Element': '[114] [< 100%RH]' , 'Tag_Number_Definition': 'Humidity sensor'}
    #element_list = element_list.append(dicc_1810, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1810])])

    dicc_1810 = {'Prop_PID_Number': '1.810_precon', 'Prop_PID_Element': '[702] [15%RH < 20%RH]' , 'Tag_Number_Definition': 'Humidity sensor'}
    #element_list = element_list.append(dicc_1810, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1810])])

    dicc_1810 = {'Prop_PID_Number': '1.810_deco', 'Prop_PID_Element': '[118] [< 100%RH]' , 'Tag_Number_Definition': 'Humidity sensor'}
    #element_list = element_list.append(dicc_1810, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1810])])


if (1.820) in element_list.Prop_PID_Number.values:
    dicc_1820 = {'Prop_PID_Number': '1.820_pro', 'Prop_PID_Element': '[114] [> 0.45m/s]' , 'Tag_Number_Definition': 'Air velocity sensor'}
    #element_list = element_list.append(dicc_1820, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1820])])

    dicc_1820 = {'Prop_PID_Number': '1.820_deco', 'Prop_PID_Element': '[118] [> 0.20m/s]' , 'Tag_Number_Definition': 'Air velocity sensor'}
    #element_list = element_list.append(dicc_1820, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1820])])

    dicc_1820 = {'Prop_PID_Number': '1.820_aer2', 'Prop_PID_Element': '[709] [> 0.45m/s]' , 'Tag_Number_Definition': 'Air velocity sensor'}
    #element_list = element_list.append(dicc_1820, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1820])])

if (1.825) in element_list.Prop_PID_Number.values:
    dicc_1825 = {'Prop_PID_Number': '1.825_leak', 'Prop_PID_Element': 'LEAK' , 'Tag_Number_Definition': 'Flow rate sensor'}
    #element_list = element_list.append(dicc_1825, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1825])])

if (1.840) in element_list.Prop_PID_Number.values:
    dicc_1840 = {'Prop_PID_Number': '1.840_prod', 'Prop_PID_Element': '[114] [>30Pa]' , 'Tag_Number_Definition': 'Differencial pressure sensor'}
    #element_list = element_list.append(dicc_1840, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1840])])

    dicc_1840 = {'Prop_PID_Number': '1.840_deco', 'Prop_PID_Element': '[118] [>15Pa]' , 'Tag_Number_Definition': 'Differencial pressure sensor'}
    #element_list = element_list.append(dicc_1840, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1840])])

    dicc_1840 = {'Prop_PID_Number': '1.840_leak', 'Prop_PID_Element': '[108] [60Pa]' , 'Tag_Number_Definition': 'Differencial pressure sensor'}
    #element_list = element_list.append(dicc_1840, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1840])])

    dicc_1840 = {'Prop_PID_Number': '1.840_emy', 'Prop_PID_Element': '[105] [<-200Pa >200Pa]' , 'Tag_Number_Definition': 'Differencial pressure sensor'}
    #element_list = element_list.append(dicc_1840, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_1840])])

#SIS Module
if (1.815) in element_list.Prop_PID_Number.values:
    dicc_sis_1 = {'Prop_PID_Number': 'DU01', 'Prop_PID_Element': 'DOSING_UNIT' , 'Tag_Number_Definition': 'Dosing unit 01'}
    #element_list = element_list.append(dicc_1840, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_sis_1])])

if (1.816) in element_list.Prop_PID_Number.values:
    dicc_sis_2 = {'Prop_PID_Number': 'DU02', 'Prop_PID_Element': 'DOSING_UNIT' , 'Tag_Number_Definition': 'Dosing unit 02'}
    #element_list = element_list.append(dicc_1840, ignore_index=True)
    element_list = pd.concat([element_list, pd.DataFrame([dicc_sis_2])])

##Analog error goes also here
#Like 1.550 analog fault, 1.555



In [1708]:
element_list.reset_index(drop=True, inplace=True)

In [1709]:
#Software Module Definition
SOFTWARE_MODE_DEFINITION = {
    "S_AAS_MAS_TCP" : "S_AAS_MAS_TCP",
    "S_AAS_MODBUS_TCP" : "S_AAS_MODBUS_TCP",
    "S_AAS_SERIAL" : "S_AAS_SERIAL",
    "S_ALARM" : "S_ALARM",
    "S_DRIVE" : "S_DRIVE",
    "S_ELMT_ON_OFF" : "S_ELMT_ON_OFF",
    "S_EMITTER_CMT" : "S_EMITTER_CMT",
    "S_FIPA" : "S_FIPA",
    "S_FLOW_METER" : "S_FLOW_METER",
    "S_GASKET" : "S_GASKET",
    "S_LEAK" : "S_LEAK",
    "S_LIMIT_HI_LO" : "S_LIMIT_HI_LO",
    "S_LIMIT_ON_OFF" : "S_LIMIT_ON_OFF",
    "S_LIMIT_SWITCH" : "S_LIMIT_SWITCH",
    "S_LINE" : "S_LINE",
    "S_MOTOR1" : "S_MOTOR1",
    "S_PLANT" : "S_PLANT",
    "S_PMS_MODBUS_ASCII" : "S_PMS_MODBUS_ASCII",
    "S_PMS_MODBUS_TCP" : "S_PMS_MODBUS_TCP",
    "S_PUMP_TEST" : "S_PUMP_TEST",
    "S_REG_CONT" : "S_REG_CONT",
    #S_REG = "S_REG"
    "S_SIS" : "S_SIS",
    "S_SKANFOG" : "SKANFOG",
    #S_TIMESYNCH = "S_TIMESYNCH"
    "S_VALVE_FLAP" : "S_VALVE_FLAP",
    #S_WEIGHT_SCALE_NG = "S_WEIGHT_SCALE_NG"
    #S_WEIGHT_SCALE = "S_WEIGHT_SCALE"
    "S_WEIGHT_SCALE2_NG" : "S_WEIGHT_SCALE2_NG",
    "S_LEADER_FOLLOWER" : "S_LEADER_FOLLOWER"
    }

In [1710]:
#element_list.index[element_list.Prop_PID_Number == 1.108].to_flat_index()

#Return the value of element
    #response = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SW_Module_Nr']
    #print(response)

#Return index position of an element given the pid number (return list, use the first element of the list ([0]))
    #index_of = element_list.index[element_list['Prop_PID_Number'] == 1.840].to_list()
    #print(index_of[0]) #[0] first element, [-1] last element

    #element_list.SW_Module_Nr[index_of[0]] = 77

#Write information in a cell:
#indice = 3
#using name
    #element_list.Prop_PID_Number[indice] = 'Change pid number'
    #element_list.Connection3_Nr[indice] = 11
#using index
    #element_list.iloc[2,10]=99 


#element_list.iloc[75,0]

#SOFT SORTING. Classifies objects just for the combination of PID elements

In [1711]:
#get value in one line of code:
#element_list.at[items, 'Connection3_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SW_Module_Nr']
#Other way:
#element_list.Connection3_Nr[items] = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SW_Module_Nr']

#Counters:
idx_drive = 0
idx_mmotor = 0
idx_reg = 0
idx_limit_hi_lo = 0
idx_elmt_on_off = 0
idx_valve_flap = 0
limit_switch = 0
sis = 0
sis_pump = 0
weight_scale2_ng = 0
leak = 0
particle_counter = 0
s_aas_mas_tcp = 0
s_fipa = 0
leader_follower = 0
air_dryer = 0


for items in element_list.index:

    response_code = element_list.at[items, 'Prop_PID_Element'] 

    match response_code:

        #Drive (3 connections)
        case ['M', 'SC', 'TSA+', 'V']:
            #Drive
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_DRIVE']
            idx_drive = idx_drive + 1 
            element_list.at[items, 'SW_Module_Nr'] = idx_drive

            #Motor Conn 1
            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION["S_MOTOR1"]
            idx_mmotor = idx_mmotor + 1
            element_list.at[items, 'Connection1_Nr'] = idx_mmotor

            #Reg Conn 2
            element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]
            idx_reg = idx_reg + 1
            element_list.at[items, 'Connection2_Nr'] = idx_reg

            #Limit hi lo conn3
            element_list.at[items, 'Connection3'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            element_list.at[items, 'Connection3_Nr'] = '?'
            
        #Motor/reg  (2 Connections)
        case ['H', 'M', 'NC'] | ['EC', 'TSA+']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION["S_MOTOR1"]
            idx_mmotor = idx_mmotor + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_mmotor

            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]
            idx_reg = idx_reg + 1
            element_list.at[items, 'Connection1_Nr'] = idx_reg

            element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            element_list.at[items, 'Connection2_Nr'] = '?'

        #Motor Simple
        case ['M', 'SC', 'V', 'XA'] | ['V', 'YC'] | ['GIA', 'K', 'Y', 'YC'] | ['GIA', 'V', 'Y', 'YC'] | ['FIA+', 'H', 'YC']: 
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION["S_MOTOR1"]
            idx_mmotor = idx_mmotor + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_mmotor

        #Limit hi lo
        case ['PDI'] | ['PDIS'] | ['TIC'] | ['FICA±'] | ['TIA±'] | ['TICA±'] | ['MIA±'] | ['SIA±'] | ['M', 'P', 'QIA+', 'XA'] | ['QIA-'] | ['QSA+'] | ['PDICA±'] | ['PSA-', 'V']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_limit_hi_lo

        #idx_elmt_on_off
        case ['M', 'P'] | ['L']:  
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_ELMT_ON_OFF']
            idx_elmt_on_off = idx_elmt_on_off + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_elmt_on_off

        #idx_valve_flap
        case ['GSA+', 'GSA-', 'V', 'Y'] | ['GSA+', 'GSA-', 'K', 'Y'] | ['V']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_VALVE_FLAP']
            idx_valve_flap = idx_valve_flap + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_valve_flap

        #limit_switch
        case ['FSA-'] | ['GZA-'] | ['GSA+'] | ['GSA-'] | ['LSA-'] : 
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_SWITCH']
            limit_switch = limit_switch + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_switch


        #Servo mousehole
        case ['GIA+', 'GIA-', 'M', 'SC', 'XA']:
            #Limit hi lo conn1
            element_list.at[items, 'SW_Module'] = "S_SERVO_MOUSEHOLE"
            #idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = '?'
        
        #Particle counter
        case ['FA-', 'QIA+', 'XA']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_PMS_MODBUS_TCP']
            particle_counter = particle_counter + 1
            element_list.at[items, 'SW_Module_Nr'] = particle_counter


        #Air Sampler MAS TCP
        case ['P', 'XA']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_AAS_MAS_TCP']
            s_aas_mas_tcp = s_aas_mas_tcp + 1
            element_list.at[items, 'SW_Module_Nr'] = s_aas_mas_tcp

        #Airdryer
        case ['T', 'XA']:
            element_list.at[items, 'SW_Module'] = 'Air dryer Munter or similar'
            air_dryer = air_dryer + 1
            element_list.at[items, 'SW_Module_Nr'] = air_dryer

        #FiPa
        case ['M', 'XA']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_FIPA']
            s_fipa = s_fipa + 1
            element_list.at[items, 'SW_Module_Nr'] = s_fipa

############################################################################################################################


        #WICA # WORKAROUD BECAUSE OF ±
        case ['WICA±']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_WEIGHT_SCALE2_NG']
            weight_scale2_ng = weight_scale2_ng + 1
            element_list.at[items, 'SW_Module_Nr'] = weight_scale2_ng

            #Limit hi lo conn1
            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'Connection1_Nr'] = idx_limit_hi_lo

            #Limit hi lo conn2
            element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_SIS"]
            element_list.at[items, 'Connection2_Nr'] = '?'


        #S_PUMP_TEST = SIS pumps: Controlled though SIS module (Unofficial)
        case ['M', 'P', 'SC']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION["S_PUMP_TEST"]
            sis_pump = sis_pump + 1  #always increment, independently from pump type
            sis_pump_airflow = sis_pump                             
            element_list.at[items, 'SW_Module_Nr'] = sis_pump_airflow #sis_pump_700 for heating plates, sis_pump_skanfog with H2O2 flow sensor (without weight scale)

            #SIS conn1
            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION["S_SIS"]
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'Connection1_Nr'] = '?'  


        #DOSING UNIT
        case 'DOSING_UNIT':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_SIS']
            sis = sis + 1
            element_list.at[items, 'SW_Module_Nr'] = sis

            #Limit hi lo Temperature Isolator
            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"] #Temperature
            element_list.at[items, 'Connection1_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.800].iloc[0]['SW_Module_Nr']

            #Limit hi lo Humidity Isolator
            element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"] #Humidity
            element_list.at[items, 'Connection2_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.810].iloc[0]['SW_Module_Nr']

            #Limit hi lo Weight Scale
            element_list.at[items, 'Connection3'] = SOFTWARE_MODE_DEFINITION["S_WEIGHT_SCALE2_NG"]#Which has a idx_limit_hi_lo used for this SIS module
            element_list.at[items, 'Connection3_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.815].iloc[0]['SW_Module_Nr']

            #S_PUMP_TEST
            element_list.at[items, 'Connection4'] = SOFTWARE_MODE_DEFINITION["S_PUMP_TEST"]# have a list of the pumps belonging to this SIS
            element_list.at[items, 'Connection4_Nr'] = '?'   

            #List of connected pumps
            #element_list.at[items, 'Connection5'] = SOFTWARE_MODE_DEFINITION["A_SIS_PUMP"]
            #idx_limit_hi_lo = idx_limit_hi_lo + 1
            #element_list.at[items, 'Connection5_Nr'] = '?'      

############################################################################################################################

        ## Artifcial sensors
        case 'LEAK':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LEAK']
            leak = leak + 1
            element_list.at[items, 'SW_Module_Nr'] = leak

            #Limit hi lo conn1
            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            element_list.at[items, 'Connection1_Nr'] = '?'

        case '[702] [17°C < 28°C]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_limit_hi_lo
        
        case '[118] [17°C < 28°C]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_limit_hi_lo

        case '[114] [15°C < 21°C]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_limit_hi_lo

        case '[708] [30°C < 40°C]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_limit_hi_lo

        case '[114] [< 100%RH]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_limit_hi_lo

        case '[702] [15%RH < 20%RH]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_limit_hi_lo

        case '[118] [< 100%RH]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_limit_hi_lo

        case '[114] [> 0.45m/s]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_limit_hi_lo

        case '[118] [> 0.20m/s]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_limit_hi_lo

        case '[709] [> 0.45m/s]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_limit_hi_lo

        case '[114] [>30Pa]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_limit_hi_lo

        case '[118] [>15Pa]': 
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_limit_hi_lo
        
        case '[108] [60Pa]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_limit_hi_lo

        case '[105] [<-200Pa >200Pa]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_limit_hi_lo

        case 'AHU01':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_REG_CONT']
            idx_reg = idx_reg + 1
            element_list.at[items, 'SW_Module_Nr'] = idx_reg
            
            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            #idx_limit_hi_lo = idx_limit_hi_lo + 1
            element_list.at[items, 'Connection1_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.800].iloc[0]['SW_Module_Nr']

            element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION['S_REG_CONT']
            #idx_reg = idx_reg + 1
            element_list.at[items, 'Connection2_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.655].iloc[0]['Connection1_Nr']

            element_list.at[items, 'Connection3'] = SOFTWARE_MODE_DEFINITION['S_REG_CONT']
            #idx_reg = idx_reg + 1
            element_list.at[items, 'Connection3_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.680].iloc[0]['Connection1_Nr']


            #print("Yikers. I don't know that one.")


In [1712]:
#Works if PID is the index
#value =(element_list.at[1.840, 'SW_Module_Nr'])
#print(value)

In [1713]:
#Update with the S_limit hi lo value
for items in element_list.index:
    
    #print(element_list.Prop_PID_Number[items])

    response_code = element_list.at[items, 'Prop_PID_Element'] 

    match response_code:
        #Drive
        case ['M', 'SC', 'TSA+', 'V']:
            #Limit hi lo conn3
            element_list.at[items, 'Connection3'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]

            #If element is 1.150, get info from 1.840, 2.150 get info from 2.840
            y840 = str("%.3f" % element_list.Prop_PID_Number[items])
            y840 = float(y840[0] + '.840')
            element_list.Connection3_Nr[items] = element_list[element_list['Prop_PID_Number'] == y840].iloc[0]['SW_Module_Nr']
            #or
            #element_list.at[items, 'Connection3_Nr'] = element_list[element_list['Prop_PID_Number'] == y840].iloc[0]['SW_Module_Nr']

            #Y.840 #only when PID is index
            #element_list.at[items, 'Connection3_Nr'] = (element_list.at[y840, 'SW_Module_Nr']) 
            

        #1.655 [H, M, NC] Control valve (Cool water control)
        case ['H', 'M', 'NC']: 
            #element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            #limit_hi_lo = limit_hi_lo + 1
            y688 = str("%.3f" % element_list.Prop_PID_Number[items])
            y688 = float(y688[0] + '.688')
            element_list.Connection2_Nr[items] = element_list[element_list['Prop_PID_Number'] == y688].iloc[0]['SW_Module_Nr']
            

        #1.680 [D, EC, TSA+]  Electric heating
        case ['EC', 'TSA+']: 
            #element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            y688 = str("%.3f" % element_list.Prop_PID_Number[items])
            y688 = float(y688[0] + '.688')
            element_list.Connection2_Nr[items] = element_list[element_list['Prop_PID_Number'] == y688].iloc[0]['SW_Module_Nr']


        case 'AHU01':
            element_list.at[items, 'Connection1_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.800].iloc[0]['SW_Module_Nr']
            element_list.at[items, 'Connection2_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.655].iloc[0]['Connection1_Nr']
            element_list.at[items, 'Connection3_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.680].iloc[0]['Connection1_Nr']


        case 'LEAK':
            element_list.at[items, 'Connection1_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SW_Module_Nr']

        

        
        #S_PUMP_TEST = SIS pumps: Controlled though SIS module (Unofficial)
        case ['M', 'P', 'SC']:
            element_list.at[items, 'Connection1_Nr'] =  element_list[element_list['Prop_PID_Number'] =='DU01'].iloc[0]['SW_Module_Nr'] #ARRAY ON PUMP INSIDE
            #element_list.at[items, 'Connection1_Nr'] =  element_list[element_list['Prop_PID_Number'] =='DU02'].iloc[0]['SW_Module_Nr'] #ARRAY ON PUMP INSIDE
        case 'DOSING_UNIT':
            element_list.at[items, 'Connection4_Nr'] = element_list[element_list['Prop_PID_Number'] =='DU01'].iloc[0]['SW_Module_Nr']   
            #element_list.at[items, 'Connection4_Nr'] = element_list[element_list['Prop_PID_Number'] =='DU02'].iloc[0]['SW_Module_Nr'] 
        case ['WICA±']:
            #Limit hi lo conn2
            #element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_SIS"]
            element_list.at[items, 'Connection2_Nr'] = element_list[element_list['Prop_PID_Number'] =='DU01'].iloc[0]['SW_Module_Nr'] 
            #element_list.at[items, 'Connection2_Nr'] = element_list[element_list['Prop_PID_Number'] =='DU02'].iloc[0]['SW_Module_Nr'] 


        


In [1714]:
if debug_mode: print(element_list.to_string())

In [1715]:
#Export to excel
"""
# create the writer object
writer = pd.ExcelWriter('Element_list_export.xlsx')
# write dataframe to excel
element_list.to_excel(writer)
# save the excel
writer.save()
# close excel
writer.close()
"""

"\n# create the writer object\nwriter = pd.ExcelWriter('Element_list_export.xlsx')\n# write dataframe to excel\nelement_list.to_excel(writer)\n# save the excel\nwriter.save()\n# close excel\nwriter.close()\n"

Init group element list

In [1716]:
#Create a new dataframe, group elements by binaries paar, later group by SW_modules, SIS, Gasket, etc...
group_element_list = pd.DataFrame()
group_element_list = pd.DataFrame(columns=['Prop_PID_Number','Tag_Number_Definition','SW_Module','SW_Module_Nr','Connection1','Connection1_Nr'])
if debug_mode: print(group_element_list)

#element_list.index[element_list.Prop_PID_Number == 1.108].to_flat_index()

#Return the value of element
#response = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SW_Module_Nr']
#print(response)

#Return index position of an element given the pid number (return list, use the first element of the list ([0]))
#index_of = element_list.index[element_list['Prop_PID_Number'] == 1.840].to_list()
#print(index_of[0]) #[0] first element, [-1] last element

#element_list.SW_Module_Nr[index_of[0]] = 77

#Write information in a cell:
#indice = 3
#using name
#element_list.Prop_PID_Number[indice] = 'Change pid number'
#element_list.Connection3_Nr[indice] = 11
#using index
#element_list.iloc[2,10]=99 


#element_list.iloc[75,0]

In [1717]:
#dicc_1825 = {'Prop_PID_Number': '1.825_leak', 'Prop_PID_Element': 'LEAK' , 'Tag_Number_Definition': 'Flow rate sensor'}
#element_list = element_list.append(dicc_1825, ignore_index=True)
#group_element_list = element_list.copy()    
#group_element_list.loc[:] = None
#group_element_list.iloc[0] = element_list.iloc[0]
#group_element_list.iloc[0] = element_list.iloc[1]
#group_element_list = pd.concat([group_element_list, element_list.loc[0]], axis="columns")
#print(element_list.iloc[1])

In [1718]:
group_element_list

,Prop_PID_Number,Tag_Number_Definition,SW_Module,SW_Module_Nr,Connection1,Connection1_Nr


In [1719]:
#LA MANDANGA
for items in element_list.index:

    response_code = element_list.at[items, 'Prop_PID_Element'] 

    match response_code:

       #Pids with 4 connections
        case 'DOSING_UNIT':
            #DOSING_UNIT: S_SIS - (1) s_LIMIT_HI_LO - (2)S_LIMIT_HI_LO - (3)S_WICA - (4)S_PUMP_TEST
            #SW_Module with all 
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            #Connection1 with all
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            #Connection2 with all
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            #Connection3 with all
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            #Connection4 with all
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1


        #Pids with 3 connections
        case ['M', 'SC', 'TSA+', 'V'] | 'AHU01':
            #['M', 'SC', 'TSA+', 'V']: S_DRIVE - (1) S_MOTOR - (2) S_REG_CONT - (3) S_LIMIT_HI_LO
            #AHU01: S_REG - (1) S_LIMIT_HI_LO - (2) S_REG_CONT - (3) S_REG_CONT 
            #SW_Module with all 
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            #Connection1 with all
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            #Connection2 with all
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            #Connection3 with all
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1

        #Pids with 2 connections
        case ['H', 'M', 'NC'] | ['EC', 'TSA+'] | ['WICA±']  :
            #['H', 'M', 'NC']: S_REG - (1) S_LIMIT_HI_LO - (2) S_REG_CONT - Cooling with Leader
            #['EC', 'TSA+']: S_REG - (1) S_LIMIT_HI_LO - (2) S_REG_CONT - heating with leader
            #['WICA±']: S_WEIGHT_SCALE - (1)S_SIS - (2)S_PUMP_TEST 
            #['M', 'P', 'SC'] S_PUMP_TEST -(1) S_SIS - (2)S_WEIGHT_SCALE
            #SW_Module with all 
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            #Connection1 with all
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            #Connection2 with all
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
        
    
        #Pump test - 1 connection
        case ['T', 'XA'] | ['M', 'P', 'SC']:
            #['T', 'XA']: S_AIR_DRYER - (1)S_LIMIT_HI_LO
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1 
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1


        #Pids with NO connection
        #Motor Simple
        case ['M', 'SC', 'V', 'XA'] | ['V', 'YC'] | ['GIA', 'K', 'Y', 'YC'] | ['GIA', 'V', 'Y', 'YC'] | ['FIA+', 'H', 'YC']: 
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1

        #Limit hi lo
        case ['PDI'] | ['PDIS'] | ['TIC'] | ['FICA±'] | ['TIA±'] | ['TICA±'] | ['MIA±'] | ['SIA±'] | ['M', 'P', 'QIA+', 'XA'] | ['QIA-'] | ['QSA+'] | ['PDICA±'] | ['PSA-', 'V']:
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1

        #elmt_on_off
        case ['M', 'P'] | ['L']:  
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1

        #valve_flap
        case ['GSA+', 'GSA-', 'V', 'Y'] | ['GSA+', 'GSA-', 'K', 'Y'] | ['V']:
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1

        #limit_switch
        case ['FSA-'] | ['GZA-'] | ['GSA+'] | ['GSA-'] | ['LSA-'] : 
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1

        #Servo mousehole
        case ['GIA+', 'GIA-', 'M', 'SC', 'XA']:
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
        
        #Particle counter
        case ['FA-', 'QIA+', 'XA']:
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1

        #Air Sampler MAS TCP
        case ['P', 'XA']:
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1

        #FiPa
        case ['M', 'XA']:
            group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1            


C:\Users\marten1\AppData\Local\Temp\ipykernel_15268\963894184.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
C:\Users\marten1\AppData\Local\Temp\ipykernel_15268\963894184.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
C:\Users\marten1\AppData\Local\Temp\ipykernel_15268\963894184.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group_element_list = group_element_list.append(element_list.loc[items], ignore_index=True) #SW_Module : Connection1
C:\Users\marten1\AppData\Local\Temp\ipyk

In [1720]:
 #LA MANDANGA II
for items in element_list.index:

    #print(items)

    #index_of =element_list.iloc[items]['Prop_PID_Number'] #Index of 
                #list of index where 1.150 is:
            #index_of = group_element_list.index[element_list.iloc[items]['Prop_PID_Element']].to_list()
    index_of = group_element_list.index[group_element_list['Prop_PID_Number'] == element_list.iloc[items]['Prop_PID_Number']].to_list()

    print(index_of)

    response_code = element_list.at[items, 'Prop_PID_Element'] 

    print(response_code)


    match response_code:

      
        #Pids with 3 connections
        case ['M', 'SC', 'TSA+', 'V'] | 'AHU01':
            #Drive with all

            #Drive witn motor - in first element [index_of[0] - from first element, copy connection1 to connection1
            #group_element_list.Connection1[index_of[0]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
            #group_element_list.Connection1_Nr[index_of[0]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr

            group_element_list.Connection1[index_of[1]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
            group_element_list.Connection1_Nr[index_of[1]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr
            group_element_list.Connection1[index_of[2]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3
            group_element_list.Connection1_Nr[index_of[2]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3

            #Motor with all - Copy motor to first position first
            #3 motors
            group_element_list.SW_Module[index_of[3]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor
            group_element_list.SW_Module_Nr[index_of[3]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.SW_Module[index_of[4]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor 
            group_element_list.SW_Module_Nr[index_of[4]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.SW_Module[index_of[5]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
            group_element_list.SW_Module_Nr[index_of[5]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number

            #3motors with others (drive, reg, limit)
            group_element_list.Connection1[index_of[3]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
            group_element_list.Connection1_Nr[index_of[3]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
            group_element_list.Connection1[index_of[4]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
            group_element_list.Connection1_Nr[index_of[4]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
            group_element_list.Connection1[index_of[5]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
            group_element_list.Connection1_Nr[index_of[5]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number


            #reg with all 

            #3 reg
            group_element_list.SW_Module[index_of[6]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor
            group_element_list.SW_Module_Nr[index_of[6]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.SW_Module[index_of[7]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor 
            group_element_list.SW_Module_Nr[index_of[7]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.SW_Module[index_of[8]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
            group_element_list.SW_Module_Nr[index_of[8]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number

            #3motors with others (drive, reg, limit)
            group_element_list.Connection1[index_of[6]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
            group_element_list.Connection1_Nr[index_of[6]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
            group_element_list.Connection1[index_of[7]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
            group_element_list.Connection1_Nr[index_of[7]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
            group_element_list.Connection1[index_of[8]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
            group_element_list.Connection1_Nr[index_of[8]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number


            #limit with all 

            #3 reg
            group_element_list.SW_Module[index_of[9]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor
            group_element_list.SW_Module_Nr[index_of[9]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.SW_Module[index_of[10]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor 
            group_element_list.SW_Module_Nr[index_of[10]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.SW_Module[index_of[11]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
            group_element_list.SW_Module_Nr[index_of[11]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number

            #3motors with others (drive, reg, limit)
            group_element_list.Connection1[index_of[9]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
            group_element_list.Connection1_Nr[index_of[9]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
            group_element_list.Connection1[index_of[10]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
            group_element_list.Connection1_Nr[index_of[10]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
            group_element_list.Connection1[index_of[11]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
            group_element_list.Connection1_Nr[index_of[11]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number

          
        #Pids with 2 connections
        case ['H', 'M', 'NC'] | ['EC', 'TSA+'] | ['WICA±']  :
            group_element_list.Connection1[index_of[0]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #SW_Module[0] - Connection1
            group_element_list.Connection1_Nr[index_of[0]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #SW_Module_Nr[0] - Connection1
            group_element_list.Connection1[index_of[1]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] 
            group_element_list.Connection1_Nr[index_of[1]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] 

            group_element_list.SW_Module[index_of[2]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] 
            group_element_list.SW_Module_Nr[index_of[2]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] 
            group_element_list.SW_Module[index_of[3]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] 
            group_element_list.SW_Module_Nr[index_of[3]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] 

            group_element_list.SW_Module[index_of[4]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor
            group_element_list.SW_Module_Nr[index_of[4]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.SW_Module[index_of[5]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor 
            group_element_list.SW_Module_Nr[index_of[5]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number

            group_element_list.Connection1[index_of[2]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] 
            group_element_list.Connection1_Nr[index_of[2]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] 
            group_element_list.Connection1[index_of[3]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] 
            group_element_list.Connection1_Nr[index_of[3]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] 

            group_element_list.Connection1[index_of[4]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #Motor
            group_element_list.Connection1_Nr[index_of[4]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.Connection1[index_of[5]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor 
            group_element_list.Connection1_Nr[index_of[5]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
    

        #pid with 1 connections, 2 elements
        case ['T', 'XA'] | ['M', 'P', 'SC']:
          #['M', 'P', 'SC'] : PUMP_TEST : SIS
          #WIKA 
            group_element_list.SW_Module[index_of[0]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
            group_element_list.SW_Module_Nr[index_of[0]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr
            group_element_list.Connection1[index_of[0]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
            group_element_list.Connection1_Nr[index_of[0]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr

            group_element_list.SW_Module[index_of[1]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor
            group_element_list.SW_Module_Nr[index_of[1]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.Connection1[index_of[1]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
            group_element_list.Connection1_Nr[index_of[1]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr
#""" 
        #Pids with 4 connections
        case 'DOSING_UNIT' :
            
            #Drive with all

            #SW <-- SW (OMITTED)
                                                #[0]    MOTOR: REG
                                                  #[0]  MOTOR: REG
                                                #[1]    MOTOR: LHL
                                                  #[1]  MOTOR: LHL
                                                #[2]
                                                  #[2]
                                                #[3]
                                                  #[3]

            #SW <-- Connection1 (OMITTED)
                                                #[0]    MOTOR: REG
                                                  #[0]  MOTOR: REG
                                                #[1]    MOTOR: REG
                                                  #[1]  MOTOR: REG
                                                #[2]
                                                  #[2]
                                                #[3]
                                                  #[3]

            ##################################################################################################################                            
            #SW Connection1<-- Connection1
            group_element_list.SW_Module[index_of[4]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor 
            group_element_list.SW_Module_Nr[index_of[4]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.SW_Module[index_of[5]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
            group_element_list.SW_Module_Nr[index_of[5]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number
            group_element_list.SW_Module[index_of[6]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor
            group_element_list.SW_Module_Nr[index_of[6]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.SW_Module[index_of[7]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor 
            group_element_list.SW_Module_Nr[index_of[7]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number

          
           #SW Connection2 <-- Connection2
            group_element_list.SW_Module[index_of[8]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
            group_element_list.SW_Module_Nr[index_of[8]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number
            group_element_list.SW_Module[index_of[9]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor
            group_element_list.SW_Module_Nr[index_of[9]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.SW_Module[index_of[10]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor 
            group_element_list.SW_Module_Nr[index_of[10]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.SW_Module[index_of[11]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
            group_element_list.SW_Module_Nr[index_of[11]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number

            #SW Connection3 <-- Connection3
            group_element_list.SW_Module[index_of[12]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
            group_element_list.SW_Module_Nr[index_of[12]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number
            group_element_list.SW_Module[index_of[13]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor
            group_element_list.SW_Module_Nr[index_of[13]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.SW_Module[index_of[14]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor 
            group_element_list.SW_Module_Nr[index_of[14]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.SW_Module[index_of[15]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
            group_element_list.SW_Module_Nr[index_of[15]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number


            #SW Connection4 <-- Connection4
            group_element_list.SW_Module[index_of[16]] = group_element_list['Connection4'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
            group_element_list.SW_Module_Nr[index_of[16]] = group_element_list['Connection4_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number
            group_element_list.SW_Module[index_of[17]] = group_element_list['Connection4'].loc[group_element_list.index[index_of[0]]] #Motor
            group_element_list.SW_Module_Nr[index_of[17]] = group_element_list['Connection4_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.SW_Module[index_of[18]] = group_element_list['Connection4'].loc[group_element_list.index[index_of[0]]] #Motor 
            group_element_list.SW_Module_Nr[index_of[18]] = group_element_list['Connection4_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
            group_element_list.SW_Module[index_of[19]] = group_element_list['Connection4'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
            group_element_list.SW_Module_Nr[index_of[19]] = group_element_list['Connection4_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number
            
            ################################################################################################################## 
            

            #SW:T-SW
            group_element_list.Connection1[index_of[0]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
            group_element_list.Connection1_Nr[index_of[0]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr
            group_element_list.Connection1[index_of[1]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
            group_element_list.Connection1_Nr[index_of[1]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr
            group_element_list.Connection1[index_of[2]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3
            group_element_list.Connection1_Nr[index_of[2]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3
            group_element_list.Connection1[index_of[3]] = group_element_list['Connection4'].loc[group_element_list.index[index_of[0]]] #Motor
            group_element_list.Connection1_Nr[index_of[3]] = group_element_list['Connection4_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
        
            ##SW Connection1 : T- Connection1
            group_element_list.Connection1[index_of[4]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
            group_element_list.Connection1_Nr[index_of[4]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr
            group_element_list.Connection1[index_of[5]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
            group_element_list.Connection1_Nr[index_of[5]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr
            group_element_list.Connection1[index_of[6]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3
            group_element_list.Connection1_Nr[index_of[6]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3
            group_element_list.Connection1[index_of[7]] = group_element_list['Connection4'].loc[group_element_list.index[index_of[0]]] #Motor
            group_element_list.Connection1_Nr[index_of[7]] = group_element_list['Connection4_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number

            ##SW Connection2 : T- Connection2
            group_element_list.Connection1[index_of[8]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
            group_element_list.Connection1_Nr[index_of[8]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr
            group_element_list.Connection1[index_of[9]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
            group_element_list.Connection1_Nr[index_of[9]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr
            group_element_list.Connection1[index_of[10]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3
            group_element_list.Connection1_Nr[index_of[10]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3
            group_element_list.Connection1[index_of[11]] = group_element_list['Connection4'].loc[group_element_list.index[index_of[0]]] #Motor
            group_element_list.Connection1_Nr[index_of[11]] = group_element_list['Connection4_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number

            ##SW Connection3 : T- Connection3
            group_element_list.Connection1[index_of[12]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
            group_element_list.Connection1_Nr[index_of[12]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr
            group_element_list.Connection1[index_of[13]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
            group_element_list.Connection1_Nr[index_of[13]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr
            group_element_list.Connection1[index_of[14]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3
            group_element_list.Connection1_Nr[index_of[14]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3
            group_element_list.Connection1[index_of[15]] = group_element_list['Connection4'].loc[group_element_list.index[index_of[0]]] #Motor
            group_element_list.Connection1_Nr[index_of[15]] = group_element_list['Connection4_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number

            ##SW Connection4 : T- Connection4
            group_element_list.Connection1[index_of[16]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
            group_element_list.Connection1_Nr[index_of[16]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr
            group_element_list.Connection1[index_of[17]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
            group_element_list.Connection1_Nr[index_of[17]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr
            group_element_list.Connection1[index_of[18]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3
            group_element_list.Connection1_Nr[index_of[18]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3
            group_element_list.Connection1[index_of[19]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor
            group_element_list.Connection1_Nr[index_of[19]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
#"""



[0]
['M', 'SC', 'V', 'XA']
[1]
['PDI']
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
['M', 'SC', 'TSA+', 'V']
[14]
['PDI']
[15]
['M', 'SC', 'V', 'XA']
[16]
['M', 'SC', 'V', 'XA']
[17]
['PDI']
[18]
['M', 'SC', 'V', 'XA']
[19]
['PDI']
[20]
['M', 'SC', 'V', 'XA']
[21]
['PDI']
[22]
['FA-', 'QIA+', 'XA']
[23]
['M', 'P']
[24]
['GSA+', 'GSA-', 'V', 'Y']
[25]
['FSA-']
[26]
['FA-', 'QIA+', 'XA']
[27]
['M', 'P']
[28]
['GSA+', 'GSA-', 'V', 'Y']
[29]
['FSA-']
[30]
['FA-', 'QIA+', 'XA']
[31]
['M', 'P']
[32]
['GSA+', 'GSA-', 'V', 'Y']
[33]
['FSA-']
[34]
['FA-', 'QIA+', 'XA']
[35]
['M', 'P']
[36]
['GSA+', 'GSA-', 'V', 'Y']
[37]
['FSA-']
[38]
['P', 'XA']
[39]
['P', 'XA']
[40]
['P', 'XA']
[41]
['GIA+', 'GIA-', 'M', 'SC', 'XA']
[42]
['GIA+', 'GIA-', 'M', 'SC', 'XA']
[43]
['GSA+', 'GSA-', 'K', 'Y']
[44]
['GSA+', 'GSA-', 'K', 'Y']
[45]
['GSA+', 'GSA-', 'K', 'Y']
[46]
['GIA', 'K', 'Y', 'YC']
[47]
['GIA', 'V', 'Y', 'YC']
[48]
['PDIS']
[49]
['GIA', 'K', 'Y', 'YC']
[50]
['PDIS']
[51]
['GSA+', 'GSA-', 'K', 'Y']
[52

In [1721]:
#list of index positions where an element is:
index_of = group_element_list.index[group_element_list['Prop_PID_Number'] == 1.150].to_list()
index_of

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [1722]:
#lenght of a list
#index_of = element_list.index[element_list['Prop_PID_Number'] == 1.150].to_list()

#print(len(index_of))
#print(index_of)

#element_list.loc[index_of].count(axis=1).max()

#element_list.notnull()


How to Get Cell Value from Pandas DataFrame?

dataframe[‘column_name’].loc[dataframe.index[row_number]]

In [1723]:
if debug_mode: 
    print(group_element_list.to_string())

In [1724]:
#Bring description of elements connected
#group_element_list['Element_Description'] = element_list['Prop_PID_Number'].apply(xlookup, args = (tnd_pid['PID'], tnd_pid['Description']));
group_element_list['Element_Description'] = group_element_list['Tag_Number_Definition']

In [1725]:
#Delete unneccessary columns
#group_element_list = group_element_list.drop(['Prop_PID_Element','Connection2','Connection2_Nr','Connection3','Connection3_Nr','Connection4','Connection4_Nr'], axis=1)
group_element_list = group_element_list.drop(['Prop_PID_Element','Connection2','Connection2_Nr','Connection3','Connection3_Nr','Connection4','Connection4_Nr'], axis=1)

In [1726]:
if debug_mode: print(group_element_list.to_string())

In [1727]:
gdrive = group_element_list.groupby(['SW_Module','SW_Module_Nr','Prop_PID_Number','Tag_Number_Definition','Connection1','Connection1_Nr'])#.agg('Element_Description')
#,'Prop_PID_Number','Tag_Number_Definition','
gdrive.first()

Element_Description
SW_Module          SW_Module_Nr Prop_PID_Number Tag_Number_Definition Connection1   Connection1_Nr                    
S_DRIVE            1            1.15            Exhaust air fan       S_LIMIT_HI_LO 24                 Exhaust air fan
                                                                      S_MOTOR1      2                  Exhaust air fan
                                                                      S_REG_CONT    1                  Exhaust air fan
S_LIMIT_HI_LO      8            1.655           Control valve         S_MOTOR1      10                   Control valve
                                                                      S_REG_CONT    2                    Control valve
...                                                                                                                ...
S_WEIGHT_SCALE2_NG 1            DU02            Dosing unit 02        S_LIMIT_HI_LO 16                  Dosing unit 02
                                                                      S_PUMP_TEST   1                   Dosing unit 02
                                                                      S_SIS         2                   Dosing unit 02
                   2            1.816           Weight scale          S_LIMIT_HI_LO 18                    Weight scale
                                                                      S_SIS         1                     Weight scale

[100 rows x 1 columns]

In [1728]:
gdrive.first().to_excel('GroupElementList.xlsx')

End _Group element list

In [1729]:
element_list.count()

Prop_PID_Number          157
Prop_PID_Element         157
Tag_Number_Definition    156
SW_Module                157
SW_Module_Nr             157
Connection1               15
Connection1_Nr            15
Connection2                8
Connection2_Nr             8
Connection3                4
Connection3_Nr             4
Connection4                2
Connection4_Nr             2
Element_Description        0
dtype: int64

In [1730]:
#Drives
element_list_drives = group_element_list[group_element_list['SW_Module'] == 'S_DRIVE']
gdrive = element_list_drives.groupby(['SW_Module','SW_Module_Nr','Tag_Number_Definition'])
gdrive.first()

,,,Prop_PID_Number,Connection1,Connection1_Nr,Element_Description
SW_Module,SW_Module_Nr,Tag_Number_Definition,,,,
S_DRIVE,1,Exhaust air fan,1.150,S_MOTOR1,2,Exhaust air fan


In [1731]:
element_list_motors = group_element_list[group_element_list['SW_Module'] == 'S_MOTOR1']
gmotor = element_list_motors.groupby(['SW_Module','SW_Module_Nr','Tag_Number_Definition'])
gmotor.first()

Prop_PID_Number  \
SW_Module SW_Module_Nr Tag_Number_Definition                          
S_MOTOR1  1            Intake air fan                         1.100   
          2            Exhaust air fan                        1.150   
          3            Exhaust air fan                        1.160   
          4            Recirculation fan                      1.200   
          5            Recirculation fan                      1.210   
          6            Recirculation fan                      1.240   
          7            Exhaust air flap                       1.550   
          8            Exhaust bypass valve                   1.555   
          9            Exhaust air flap                       1.560   
          10           Control valve                          1.655   
          11           Electric heating                       1.680   
          12           Needle valve compressed air            1.707   
          13           Needle valve compressed air            1.717   
          14           Needle valve compressed air            1.727   
          15           Flow rate sensor                       1.825   
          16           Recirculation fan                      2.200   
          17           Recirculation fan                      2.210   
          18           Needle valve compressed air            2.707   
          19           Needle valve compressed air            2.717   
          20           Needle valve compressed air            2.727   

                                                   Connection1  \
SW_Module SW_Module_Nr Tag_Number_Definition                     
S_MOTOR1  1            Intake air fan                     None   
          2            Exhaust air fan                 S_DRIVE   
          3            Exhaust air fan                    None   
          4            Recirculation fan                  None   
          5            Recirculation fan                  None   
          6            Recirculation fan                  None   
          7            Exhaust air flap                   None   
          8            Exhaust bypass valve               None   
          9            Exhaust air flap                   None   
          10           Control valve                S_REG_CONT   
          11           Electric heating             S_REG_CONT   
          12           Needle valve compressed air        None   
          13           Needle valve compressed air        None   
          14           Needle valve compressed air        None   
          15           Flow rate sensor                   None   
          16           Recirculation fan                  None   
          17           Recirculation fan                  None   
          18           Needle valve compressed air        None   
          19           Needle valve compressed air        None   
          20           Needle valve compressed air        None   

                                                    Connection1_Nr  \
SW_Module SW_Module_Nr Tag_Number_Definition                         
S_MOTOR1  1            Intake air fan                          NaN   
          2            Exhaust air fan                       1.000   
          3            Exhaust air fan                         NaN   
          4            Recirculation fan                       NaN   
          5            Recirculation fan                       NaN   
          6            Recirculation fan                       NaN   
          7            Exhaust air flap                        NaN   
          8            Exhaust bypass valve                    NaN   
          9            Exhaust air flap                        NaN   
          10           Control valve                         2.000   
          11           Electric heating                      3.000   
          12           Needle valve compressed air             NaN   
          13           Needle valve compressed air      

In [1732]:
#AHU01
element_list_ahu01 = group_element_list[group_element_list['SW_Module'] == 'AHU01']
gahu01 = element_list_ahu01.groupby(['SW_Module','SW_Module_Nr','Tag_Number_Definition'])
gahu01.first()
    

,,,Prop_PID_Number,Connection1,Connection1_Nr,Element_Description
SW_Module,SW_Module_Nr,Tag_Number_Definition,,,,


In [1733]:
#Valve
element_list_valve_flap = group_element_list[group_element_list['SW_Module'] == 'S_VALVE_FLAP']
gvalve = element_list_valve_flap.groupby(['SW_Module','SW_Module_Nr','Tag_Number_Definition'])
gvalve.first()

Prop_PID_Number  \
SW_Module    SW_Module_Nr Tag_Number_Definition                                   
S_VALVE_FLAP 1            3-way valve                                     1.305   
             2            3-way valve                                     1.315   
             3            3-way valve                                     1.325   
             4            3-way valve                                     1.335   
             5            Supply air flap                                 1.500   
             6            Flap                                            1.510   
             7            Flap                                            1.520   
             8            Flap                                            1.570   
             9            Control valve compressed air                    1.706   
             10           Control valve compressed air                    1.716   
             11           Control valve compressed air                    1.726   
             12           Control valve compressed air                    1.746   
             13           H2O2 LC valve                                   1.831   
             14           Control valves including 3-way valve            1.860   
             15           Pressure sensor                                 1.861   
             16           Control valve compressed air                    2.706   
             17           Control valve compressed air                    2.716   

                                                               Connection1  \
SW_Module    SW_Module_Nr Tag_Number_Definition                              
S_VALVE_FLAP 1            3-way valve                                 None   
             2            3-way valve                                 None   
             3            3-way valve                                 None   
             4            3-way valve                                 None   
             5            Supply air flap                             None   
             6            Flap                                        None   
             7            Flap                                        None   
             8            Flap                                        None   
             9            Control valve compressed air                None   
             10           Control valve compressed air                None   
             11           Control valve compressed air                None   
             12           Control valve compressed air                None   
             13           H2O2 LC valve                               None   
             14           Control valves including 3-way valve        None   
             15           Pressure sensor                             None   
             16           Control valve compressed air                None   
             17           Control valve compressed air                None   

                                                               Connection1_Nr  \
SW_Module    SW_Module_Nr Tag_Number_Definition                                 
S_VALVE_FLAP 1            3-way valve                                    None   
             2            3-way valve                                    None   
             3            3-way valve                                    None   
             4            3-way valve                                    None   
             5            Supply air flap                                None   
             6            Flap                                           None   
             7            Flap                                           None   
             8            Flap                                           None   
             9            Control valve compressed air                   None   
             10           Control valve compressed air                   None   
             11           Contr

In [1734]:
#Element on off
element_list_elmt_on_off = group_element_list[group_element_list['SW_Module'] == 'S_ELMT_ON_OFF']
gelmt_on_off = element_list_elmt_on_off.groupby(['SW_Module','SW_Module_Nr','Tag_Number_Definition'])
gelmt_on_off.first()

Prop_PID_Number Connection1  \
SW_Module     SW_Module_Nr Tag_Number_Definition                                
S_ELMT_ON_OFF 1            Vacuum pump                      1.304        None   
              2            Vacuum pump                      1.314        None   
              3            Vacuum pump                      1.324        None   
              4            Vacuum pump                      1.334        None   
              5            Circulating pump                 1.651        None   
              6            Lighting                         1.990        None   
              7            Alarm lights                     1.991        None   
              8            Vacuum pump                      2.304        None   
              9            Lighting                         2.990        None   

                                                 Connection1_Nr  \
SW_Module     SW_Module_Nr Tag_Number_Definition                  
S_ELMT_ON_OFF 1            Vacuum pump                     None   
              2            Vacuum pump                     None   
              3            Vacuum pump                     None   
              4            Vacuum pump                     None   
              5            Circulating pump                None   
              6            Lighting                        None   
              7            Alarm lights                    None   
              8            Vacuum pump                     None   
              9            Lighting                        None   

                                                 Element_Description  
SW_Module     SW_Module_Nr Tag_Number_Definition                      
S_ELMT_ON_OFF 1            Vacuum pump                  Vacuum pump   
              2            Vacuum pump                  Vacuum pump   
              3            Vacuum pump                  Vacuum pump   
              4            Vacuum pump                  Vacuum pump   
              5            Circulating pump         Circulating pump  
              6            Lighting                        Lighting   
              7            Alarm lights                Alarm lights   
              8            Vacuum pump                  Vacuum pump   
              9            Lighting                        Lighting

In [1735]:
element_list_limit_switch = group_element_list[group_element_list['SW_Module'] == 'S_LIMIT_SWITCH']
glimit_switch = element_list_limit_switch.groupby(['SW_Module','SW_Module_Nr','Tag_Number_Definition'])
glimit_switch.first()

Prop_PID_Number  \
SW_Module      SW_Module_Nr Tag_Number_Definition                    
S_LIMIT_SWITCH 1             Flow-Switch                     1.308   
               2             Flow-Switch                     1.318   
               3             Flow-Switch                     1.328   
               4            Flow-Switch                      1.338   
               5            Flow Switch H2O2                 1.705   
               6            Flow Switch H2O2                 1.715   
               7            Flow Switch H2O2                 1.725   
               8            Safety door                      1.901   
               9            Safety door                      1.902   
               10           Safety door                      1.903   
               11           Safety door                      1.904   
               12           Safety door                      1.905   
               13           Safety door                      1.906   
               14           Service cover                    1.911   
               15           Service cover                    1.912   
               16           Service cover                    1.913   
               17           Service cover                    1.914   
               18           Service cover                    1.915   
               19           Service cover                    1.916   
               20           Service cover                    1.917   
               21           Service cover                    1.918   
               22           Service cover                    1.919   
               23           Mousehole door                   1.921   
               24           Mousehole door                   1.922   
               25           Mousehole door                   1.923   
               26           Mousehole door                   1.924   
               27           Safety windows                   1.931   
               28           Safety windows                   1.932   
               29           Safety front door                1.941   
               30           Safety front door                1.942   
               31           Safety front door                1.943   
               32           Safety front door                1.944   
               33           Safety front door                1.945   
               34           Safety front door                1.946   
               35           Laser Scanner                    1.961   
               36           Laser Scanner                    1.962   
               37           Laser Scanner                    1.963   
               38            Flow-Switch                     2.308   
               39           Flow Switch H2O2                 2.705   
               40           Flow Switch H2O2                 2.715   
               41           Flow Switch H2O2                 2.725   

                                                  Connection1 Connection1_Nr  \
SW_Module      SW_Module_Nr Tag_Number_Definition                              
S_LIMIT_SWITCH 1             Flow-Switch                 None           None   
               2             Flow-Switch                 None           None   
               3             Flow-Switch                 None           None   
               4            Flow-Switch                  None           None   
               5            Flow Switch H2O2             None           None   
               6            Flow Switch H2O2             None           None   
               7            Flow Switch H2O2             None           None   
               8            Safety door                  None           None   
               9            Safety door                  None           None   
               10           Safety door                  None           None   
               11           Safety door                  None           None   


In [1736]:
element_list_limit_hi_lo = group_element_list[group_element_list['SW_Module'] == 'S_LIMIT_HI_LO']
glimit_hi_lo = element_list_limit_hi_lo.groupby(['SW_Module','SW_Module_Nr','Tag_Number_Definition'])
glimit_hi_lo.first()
#print(element_list_limit_hi_lo.to_string())
#element_list_limit_hi_lo.count()

Prop_PID_Number  \
SW_Module     SW_Module_Nr Tag_Number_Definition                                         
S_LIMIT_HI_LO 1            Diferential pressure sensor                           1.108   
              2            Diferential pressure sensor                           1.158   
              3            Diferential pressure sensor                           1.208   
              4            Diferential pressure sensor                           1.218   
              5            Diferential pressure sensor                           1.248   
              6            Diferential pressure sensor                           1.558   
              7            Diferential pressure sensor                           1.568   
              8            Control valve                                         1.655   
                           Electric heating                                      1.680   
                           Temperature sensor                                    1.688   
              10           Flow switch compressed air                            1.708   
              12           Flow switch compressed air                            1.718   
              14           Flow switch compressed air                            1.728   
              15           Dosing unit 01                                         DU01   
                           Dosing unit 02                                         DU02   
                           Leader/Follower AHU 01 1.800 - (1.655-1.680           AHU01   
                           Temperature sensor                                    1.800   
              16           Dosing unit 01                                         DU01   
                           Dosing unit 02                                         DU02   
                           Humidity sensor                                       1.810   
              17           Weight scale                                          1.815   
              18           Weight scale                                          1.816   
              19           Air speed sensor                                      1.820   
              20           H2O2 LC sensor                                        1.830   
              21           H2O2 HC sensor                                        1.832   
              22           TLV H2O2 LC Sensor                                    1.833   
              23           TLV H2O2 LC Sensor                                    1.834   
              24           Differential pressure sensor                          1.840   
                           Exhaust air fan                                       1.150   
              25           Differential pressure sensor                          1.842   
              26           Pressure reduction 1                                  1.853   
              27           Inflatable Gasket                                     1.865   
              28           Inflatable Gasket                                     1.866   
              29           Inflatable Gasket                                     1.867   
              30           Inflatable Gasket                                     1.868   
              31           Inflatable Gasket                                     1.869   
              32           Inflatable Gasket                                     1.870   
              33           Diferential pressure sensor                           2.208   
              34           Diferential pressure sensor                           2.218   
              36           Flow switch compressed air                            2.708   
              38           Flow switch compressed air                            2.718   
              40           Flow switch compressed air                            2.728   

                                                                               Connection1  \
SW_Module     SW

In [1737]:
element_list_leak = group_element_list[group_element_list['SW_Module'] == 'S_LEAK']
print(element_list_leak.to_string())

Empty DataFrame
Columns: [Prop_PID_Number, Tag_Number_Definition, SW_Module, SW_Module_Nr, Connection1, Connection1_Nr, Element_Description]
Index: []


In [1738]:
#Particle counter modbus tcp
element_list_pms_modbus_tcp = group_element_list[group_element_list['SW_Module'] == 'S_PMS_MODBUS_TCP']
print(element_list_pms_modbus_tcp.to_string())


    Prop_PID_Number Tag_Number_Definition         SW_Module SW_Module_Nr Connection1 Connection1_Nr Element_Description
22            1.300      Particle counter  S_PMS_MODBUS_TCP            1        None           None    Particle counter
26            1.310      Particle counter  S_PMS_MODBUS_TCP            2        None           None    Particle counter
30            1.320      Particle counter  S_PMS_MODBUS_TCP            3        None           None    Particle counter
34            1.330      Particle counter  S_PMS_MODBUS_TCP            4        None           None    Particle counter
155           2.300      Particle counter  S_PMS_MODBUS_TCP            5        None           None    Particle counter


In [1739]:
#Air sampler modbus
element_list_aas_mas_tcp = group_element_list[group_element_list['SW_Module'] == 'S_AAS_MAS_TCP']
print(element_list_aas_mas_tcp.to_string())

    Prop_PID_Number Tag_Number_Definition      SW_Module SW_Module_Nr Connection1 Connection1_Nr Element_Description
38            1.350           Air sampler  S_AAS_MAS_TCP            1        None           None         Air sampler
39            1.360           Air sampler  S_AAS_MAS_TCP            2        None           None         Air sampler
40            1.370           Air sampler  S_AAS_MAS_TCP            3        None           None         Air sampler
158           2.350           Air sampler  S_AAS_MAS_TCP            4        None           None         Air sampler


SIS

In [1740]:
element_list_a_sis_pump = group_element_list[group_element_list['SW_Module'] == 'S_SIS']
ga_sis_pump = element_list_a_sis_pump.groupby(['SW_Module','SW_Module_Nr','Tag_Number_Definition'])
ga_sis_pump.first()

Prop_PID_Number  \
SW_Module SW_Module_Nr Tag_Number_Definition                   
S_SIS     1            Dosing unit 01                   DU01   
                       Peristaltic pump                1.700   
                       Weight scale                    1.815   
          2            Dosing unit 02                   DU02   

                                                     Connection1  \
SW_Module SW_Module_Nr Tag_Number_Definition                       
S_SIS     1            Dosing unit 01              S_LIMIT_HI_LO   
                       Peristaltic pump              S_PUMP_TEST   
                       Weight scale           S_WEIGHT_SCALE2_NG   
          2            Dosing unit 02              S_LIMIT_HI_LO   

                                              Connection1_Nr  \
SW_Module SW_Module_Nr Tag_Number_Definition                   
S_SIS     1            Dosing unit 01                     15   
                       Peristaltic pump                    1   
                       Weight scale                        1   
          2            Dosing unit 02                     15   

                                             Element_Description  
SW_Module SW_Module_Nr Tag_Number_Definition                      
S_SIS     1            Dosing unit 01             Dosing unit 01  
                       Peristaltic pump         Peristaltic pump  
                       Weight scale                 Weight scale  
          2            Dosing unit 02             Dosing unit 02

In [1741]:
#element_list_motors = element_list[element_list['SW_Module'] == 'S_MOTOR1']
#print(element_list_motors.to_string())

In [1742]:
#Group by SW_Module
#el_sortby_sw_module = element_list.groupby(['SW_Module']).agg(list).copy()
#print(el_sortby_sw_module.to_string())

In [1743]:
#Group by pdi number again
#this time
#Group by PID number
#element_list = element_list.groupby(['Prop_PID_Number']).agg(list).copy()

In [1744]:
#print(element_list.to_string())

In [1745]:
#list_1100 = element_list.loc[1.100].values.tolist()
#list_1100[0].sort()

In [1746]:
#print(list_1100[0])

end_new code

In [1747]:
#Remove duplicates - WARNING: It removes duplicates alarms as 1.310 Particle counter 0.5 um and 5.0um (maybe add exception when is an PCounter and duplicates elements)

#df_data_excel_uniques = df_data_excel.drop_duplicates()

In [1748]:
#df_data_excel_uniques.count()

In [1749]:
#gkk = df_data_excel_uniques.groupby(['Prop_PID_Number', 'Prop_PID_Element'])
#gkk.first()

In [1750]:
#Get uniques values from the PID Elements (the columns for the matrix)
#df_unique_elememts = df_data_excel.Prop_PID_Element.unique()
#df_unique_elememts.sort()
#print(df_unique_elememts)




#Export Unique elements to excel file - Once


In [1751]:
#Create a dataframe containing the elements
#pd_automation_code = pd.DataFrame(df_unique_elememts)
#Create the writer object
#automation_code = pd.ExcelWriter('AutomationCode.xlsx')
#Write dataframe to excel file
#pd_automation_code.to_excel(automation_code)
#Save excel 
#automation_code.save()
#Close excel
#automation_code.close()


In [1752]:
#Sort by PID number
#df_data_excel_sorted = df_data_excel_uniques.sort_values(by='Prop_PID_Number')

In [1753]:
#Set PID number as multi-index (group by does better)
#df_data_excel_sorted.set_index(['Prop_PID_Number','Prop_PID_Element'])

In [1754]:
#df_data_excel_sorted.head(1000)

In [1755]:
#Add column, load data from Automation Code and mix
#df_data_excel_sorted['Automation Code'] = ''


In [1756]:
#df_data_excel_sorted

In [1757]:
#Load Automation Code excel file
#df_automation_code = pd.read_excel('AutomationCode.xlsx')
#df_automation_code.head()

In [1758]:
#Lookup
#df = df_data_excel
#df['Automation_Code'] = ''
#print (df)

In [1759]:
# Deprecated df['Automation Code'] = df.set_index('Prop_PID_Element').lookup(df_automation_code.Prop_PID_Element, df_automation_code.Automation_Code)

In [1760]:
#pd.melt(df,)
#print(df.combine_first(df_automation_code))

In [1761]:
#documents = [['SW_MODULE', 'SW_MODULE_NR'], ['CONN1', 'CONN1_NR'], ['CONN2', 'CONN2_NR'], ['CONN3', 'CONN3_NR'], 'AM', 'HW']
#documents = ['EL', 'AM', 'HW', 'FN']

In [1762]:
#
#df[df_unique_elememts] = pd.NaT
#documents = ['EL', 'AM', 'HW', 'FN']
#df[documents] = pd.NaT
#print(df.to_string())

ALARM MATRIX

In [1763]:
df_am = pd.read_excel('AM.xlsm')

df_am


,Unnamed: 0,Unnamed: 1
0,MSRDok,NaN
1,NaN,NaN
2,AM - Alarm-Matrix EN,NaN
3,NaN,NaN
4,Customer,NaN
5,User,NaN
6,Identification,NaN
7,NaN,NaN
8,Project name,Vorlagen für MSR GAMP Dokumente
9,Revision date,16.02.2022


In [1764]:
df_am = pd.read_excel('AM.xlsm', sheet_name="Alarm Matrix")

c:\Users\marten1\Anaconda3\envs\threeten\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [1765]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', 1000)
#pd.set_option('display.colheader_justify', 'center')
#pd.set_option('display.precision', 3)

#display(df_am)